In [1]:
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from mymodule import PipeLine

In [12]:
# グリッドサーチの関数
def grid_serch_cv(pipe, validation, param_range, param_grid, model):
    pack = validation()
    param_range = param_range
    param_grid = param_grid

    gs = GridSearchCV(estimator=model(), 
                    param_grid=param_grid,  # 設定した候補を代入
                    scoring='accuracy', 
                    refit=True,
                    cv=10,
                    n_jobs=-1)
    gs.fit(pack[0], pack[2])  # x_train = pack[0], y_train = pack[2]

    valid, model = [validation.__name__, model]
    params = gs.best_params_
    model = pipe.training(valid=valid, Model=model ,params=params)
    return model, gs.best_params_

グリッドサーチ<span style = "color: orange;">なし</span>の場合

In [13]:
df = pd.read_csv('./data/train.csv')

pipe = PipeLine()
pipe.viewer = False
pipe(df)
model = pipe.training(valid='fold_out_split', Model=SVC)

,accuracy_score,precision_score,recall_score,f1_score
train,0.750557,0.774892,0.748954,0.761702
test,0.704663,0.773913,0.741667,0.757447


グリッドサーチ<span style = "color: orange;">あり</span>の場合

In [6]:
# グリッドサーチの探索候補
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10, 1000]
param_grid = [{'C': param_range,  # 正則化パラメータの係数を探索
               'kernel': ['linear']},  # Linearのカーネル
              {'C': param_range, 
               'gamma': param_range,  # カーネル係数を探索
               'kernel': ['rbf']}]  # rbfのカーネル
# 使用するモデル
model = SVC  # クラスの()いらない

In [7]:
# PipeLine
pipe = PipeLine()
pipe.viewer = False
pipe(df)
# PipeLineクラスのfold_out_splitインスタンスを生成しとく
validation = pipe.fold_out_split  # 関数の()いらない
# グリッドサーチの出番
model, best_params = grid_serch_cv(pipe, validation, param_range, param_grid, model)

,accuracy_score,precision_score,recall_score,f1_score
train,0.817372,0.861751,0.782427,0.820175
test,0.813472,0.850000,0.850000,0.850000


精度が向上していることがわかる<br>
※ もともと精度が高いデータセットの場合<span style = "color: orange;">精度が下がる場合がある</span><br>
（パラメータ検証のためデータセットを削っているためと思われ)
>
自作した関数では最適なモデルとパラメータを出力するようになってる


In [11]:
print(best_params)
model

{'C': 0.1, 'kernel': 'linear'}


SVC(C=0.1, kernel='linear')